*Imports:*

In [1]:
import serial
import cv2
import numpy as np
import time
from ultralytics import YOLO
import torch
from typing import Tuple


**Main Code Definitions:**

In [2]:
def clean_and_parse_data(raw_data: bytes) -> dict:
    # Decode the bytes to string, ignoring errors
    decoded_data = raw_data.decode('ascii', errors='ignore').strip()
    split_decoded_data = decoded_data.split(',')

    data_dict = {}
    for data_point in split_decoded_data:
        curr = data_point.split('=')
        if len(curr) == 2:  # Ensure that both key and value are present
            data_dict[curr[0].strip()] = curr[1].strip()  # Strip whitespace from keys and values
    return clean_uart_data(data_dict)

def clean_uart_data(data_dict: dict) -> dict:
    cleaned_dict = {}
    for key, value in data_dict.items():
        # Remove null characters from key and value
        clean_key = key.replace('\x00', '').strip()
        clean_value = value.replace('\x00', '').strip()
        
        # Add the cleaned key-value pair to the new dictionary if key is not empty
        if clean_key:
            cleaned_dict[clean_key] = clean_value
            
    return cleaned_dict

def find_center_of_frame(frame):
    height, width = frame.shape[:2]
    return width // 2, height // 2

def calculate_box_area(x1, y1, x2, y2):
    return (x2 - x1) * (y2 - y1)

def find_box_center(x1, y1, x2, y2):
    return int((x1 + x2) / 2), int((y1 + y2) / 2)

**Main Code:**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

ser = serial.Serial(
    port='COM3',  
    baudrate=9600,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS,
    timeout=0
)

# Define the window size for the moving average & other parameters
WINDOW_SIZE = 1
x_values = np.zeros(WINDOW_SIZE)
y_values = np.zeros(WINDOW_SIZE)
index = 0

# Small waiting time to wait for everything in the STM to init
time.sleep(3)

# Load the model and move it to GPU
model = YOLO('TennisBestColored.pt', task='detect').to(device)
cap = cv2.VideoCapture(1)

cap.set(cv2.CAP_PROP_FPS, 30)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)
actual_fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Camera framerate set to: {actual_fps}")

ret, frame = cap.read()
set_point_x, set_point_y = find_center_of_frame(frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if frame could not be read

    # Convert frame to RGB (YOLO expects RGB input)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run inference on GPU
    results = model(frame_rgb, device=device)

    largest_area = 0
    largest_box_center = None

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls = box.cls[0].cpu().numpy()

            if conf < 0.5:
                continue
            
            area = calculate_box_area(x1, y1, x2, y2)
            
            if area < 100:
                continue
            
            if area > largest_area:
                largest_area = area
                largest_box_center = find_box_center(x1, y1, x2, y2)
            
            label = f"{model.names[int(cls)]} {conf:.2f}"
            color = (0, 255, 0)  # Green color for bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    if largest_box_center:
        cv2.circle(frame, largest_box_center, 5, (0, 0, 255), -1)  # Red dot
        cv2.putText(frame, f"Center: {largest_box_center}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
    cv2.circle(frame, (set_point_x, set_point_y), 5, (255, 0, 0), -1)  # Blue dot
    cv2.putText(frame, f"Set Point: ({set_point_x}, {set_point_y})", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    if largest_box_center:
        # Update the moving average arrays
        x_values[index] = largest_box_center[0]
        y_values[index] = largest_box_center[1]
        
        # Calculate the moving averages
        smoothed_x = int(np.mean(x_values))
        smoothed_y = int(np.mean(y_values))
        
        # Update the index for circular buffer
        index = (index + 1) % WINDOW_SIZE
        
        diff_x = smoothed_x - set_point_x
        diff_y = smoothed_y - set_point_y
        cv2.putText(frame, f"Difference: ({diff_x}, {diff_y})", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        str_to_send = f"a{smoothed_x:03d}{smoothed_y:03d}"
    else:
        str_to_send = "a400300"

    ser.write(str_to_send.encode())
    ser.flush()
    print(str_to_send)

    raw_data = ser.readline()
    parsed_data = clean_and_parse_data(raw_data)
    print(parsed_data)

    # Draw the dictionary values on the frame
    y_offset = 0
    for key, value in parsed_data.items():
        text = f"{key}: {value}"
        if key == "Base Proj Point": y_offset = 120
        elif key == "Tilt Proj Point": y_offset = 150
        elif key == "Base PID Out": y_offset = 180
        elif key == "Tilt PID Out": y_offset = 210
        cv2.putText(frame, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    text = "Sent: " + str_to_send
    cv2.putText(frame, text, (10, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)

    cv2.imshow("YOLOv8 Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
ser.close()

Using device: cuda
Camera framerate set to: 30.0

0: 480x640 (no detections), 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


In [14]:
ser.close()

In [15]:
cap.release()
cv2.destroyAllWindows()

In [33]:
import torch
from ultralytics import YOLO
import os

def export_yolo_to_onnx(pt_file, onnx_file):
    # Load the model
    model = YOLO(pt_file)

    # Export the model
    try:
        model.export(format="onnx", opset=12, simplify=True)
        print(f"Model exported successfully to {onnx_file}")
    except Exception as e:
        print(f"Error exporting model: {str(e)}")

# Specify the input .pt file and output .onnx file
pt_file = "BestTennis.pt"
onnx_file = "BestTennis.onnx"

# Call the function
export_yolo_to_onnx(pt_file, onnx_file)

Ultralytics YOLOv8.2.80  Python-3.11.2 torch-2.3.0+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'BestTennis.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnxslim>=0.1.31'] not found, attempting AutoUpdate...

requirements: AutoUpdate success  3.4s, installed 1 package: ['onnxslim>=0.1.31']
requirements:  Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.1 opset 12...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success  6.5s, saved as 'BestTennis.onnx' (11.7 MB)

Export complete (10.4s)
Results saved to C:\Users\sandr\OneDrive\Desktop\Junior Project\PythonProgram
Predict:         yolo predict task=detect model=BestTennis.onnx imgsz=640  
Validate:        yolo val task=detect model=BestTennis.onnx imgsz=640 data=/content/Tennis-Ball-Obj-Det-1/

In [8]:
import serial.tools.list_ports

def list_serial_ports():
    # Get a list of all available serial ports
    ports = serial.tools.list_ports.comports()

    if not ports:
        print("No serial ports found.")
    else:
        print("Available serial ports:")
        for port in ports:
            print(f"- {port.device}: {port.description}")

# Call the function to list serial ports
list_serial_ports()

Available serial ports:
- COM3: Silicon Labs CP210x USB to UART Bridge (COM3)
